# Задача: Анализ эффективности нового алгоритма рекомендаций в условиях асимметричного и зашумленного распределения
### Контекст:
Крупная стриминговая платформа (аналог YouTube или Twitch) разработала новый алгоритм рекомендаций контента (алгоритм B), который, как ожидается, должен увеличить среднее время просмотра на пользователя по сравнению со старым алгоритмом (A).

Было проведено A/B-тестирование:

- **Группа A (Контрольная)**: 5000 пользователей, старый алгоритм.

- **Группа B (Тестовая)**: 5000 пользователей, новый алгоритм.

По итогам двухнедельного теста вы собрали данные по ключевой метрике — **«Общее время просмотра на пользователя за тест (в минутах)»**.

**Данные и их особенности:**

**Не нормальность:** Распределение метрики в обеих группах сильно отличается от нормального. Оно имеет типичный для поведенческих данных "длинный хвост":

- Большинство пользователей смотрят немного (пик в районе 0-50 минут).

- Небольшая группа "супер-зрителей" смотрит очень много (сотни и тысячи минут), формируя длинный правый хвост распределения.

**Асимметрия влияния:** Новый алгоритм B целенаправленно меняет опыт именно активных пользователей, в то время как для "пассивных" зрителей разница неочевидна. Это означает, что различие между группами может быть не сдвигом всего распределения, а изменением в правом хвосте.

**Наличие выбросов:** В данных присутствуют экстремальные выбросы (например, пользователи, которые оставляли видео фоном на всю ночь).

Вам предоставлен датасет _ab_test_results.csv_ со следующими колонками:

 - _user_id_

 - _group (A или B)_

 - _total_watch_time_min_

### Задания:


- **Проведите тест Манна-Уитни (Mann-Whitney U test)**. Сформулируйте нулевую и альтернативную гипотезы. Интерпретируйте p-value. Можно ли на его основе сделать однозначный вывод о превосходстве алгоритма B?

- Чтобы исследовать "асимметричное влияние", проведите анализ разниц не только в центре распределения, но и в хвостах. **Рассчитайте и визуализируйте разницу между группами для 75-го, 90-го и 95-го процентилей**.

- **Сформулируйте итоговый вывод:**

    - Эффективен ли новый алгоритм B?

    - На какую именно группу пользователей он оказывает наибольшее влияние?

    - Какой метод анализа оказался наиболее информативным в данном случае и почему?

    - Дайте рекомендацию о возможности запуска алгоритма на всю аудиторию.



In [20]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [22]:
# Load CSV
df = pd.read_csv('ab_test_results.csv')

group_a_data = df[df['group'] == 'A']['total_watch_time_min']
group_b_data = df[df['group'] == 'B']['total_watch_time_min']

# Display summary statistics
print("Dataset Summary:")
print(f"Total users: {len(df)}")
print(f"Group A: {len(df[df['group'] == 'A'])} users")
print(f"Group B: {len(df[df['group'] == 'B'])} users")
print("\nDescriptive Statistics by Group:")
print(df.groupby('group')['total_watch_time_min'].describe())

print("\nKey Percentiles by Group:")
percentiles = [0.75, 0.9, 0.95]
for p in percentiles:
    p_val = p * 100
    a_val = np.percentile(group_a_data, p_val)
    b_val = np.percentile(group_b_data, p_val)
    diff = b_val - a_val
    print(f"{p_val:.0f}th percentile: A={a_val:.1f}, B={b_val:.1f}, Difference={diff:.1f}")

# Test the differences
from scipy import stats
mw_stat, mw_p = stats.mannwhitneyu(
    group_a_data,
    group_b_data,
    alternative= 'two-sided' #'less' #'greater' #'two-sided'
)

print(f"\nMann-Whitney U Test: p-value = {mw_p:.6f}")

Dataset Summary:
Total users: 10000
Group A: 5000 users
Group B: 5000 users

Descriptive Statistics by Group:
        count       mean         std  min        25%        50%        75%  \
group                                                                        
A      5000.0  56.347878  262.313288  0.0  14.729144  26.834960  44.946309   
B      5000.0  71.477408  408.519252  0.0  15.383838  27.352416  46.352279   

                max  
group                
A       8130.547119  
B      11670.621483  

Key Percentiles by Group:
75th percentile: A=44.9, B=46.4, Difference=1.4
90th percentile: A=71.5, B=76.2, Difference=4.7
95th percentile: A=119.3, B=153.6, Difference=34.3

Mann-Whitney U Test: p-value = 0.045342


In [25]:
# Загрузка данных
df = pd.read_csv('ab_test_results.csv')

# Разделение на группы
group_a = df[df['group'] == 'A']['total_watch_time_min']
group_b = df[df['group'] == 'B']['total_watch_time_min']

#1 Основная статистика
print("Анализ A/B Теста: время просмотра")
# Сводка по датасету
total_users = len(df)
group_a_count = len(group_a)
group_b_count = len(group_b)
print(f"\nDataset Summary:")
print(f"Total users: {total_users}")
print(f"Group A: {len(df[df['group'] == 'A'])} users")
print(f"Group B: {len(df[df['group'] == 'B'])} users")

# Описательная статистика
stats_a = group_a.describe()
stats_b = group_b.describe()

# Создаем DataFrame для статистики
stats_df = pd.DataFrame({
    'A': stats_a,
    'B': stats_b
}).T
stats_df.index.name = 'Group' # Добавляем названия статистик
stats_df.columns = ['count', 'mean', 'std', 'min', '25%', 'mediana', '75%', 'max']
print(f"\nDescriptive Statistics by Group:")
print(stats_df.to_string(float_format='%.2f'))

# 2 Анализ ключевых процентилей
print(f"Key Percentiles by Group:")
percentiles = [75, 90, 95]
for p in percentiles:
    a_p = np.percentile(group_a, p)
    b_p = np.percentile(group_b, p)
    diff = b_p - a_p
    print(f"{p}й процентиль: A={a_p:.1f}, B={b_p:.1f}, Difference={diff:.1f}")

# 3Статистический тест Манна-Уитни
mw_stat, mw_p = stats.mannwhitneyu(group_a, group_b, alternative='two-sided')
print(f"\nMann-Whitney U Test: p-value = {mw_p:.6f}")

# Интерпретация p-значения
print("\nИнтерпретация:")
if mw_p < 0.05:
    print("Статистически значимое различие между группами (p < 0.05)")
else:
    print("Нет статистически значимого различия между группами (p ≥ 0.05)")

# Проверяем, есть ли пользователи с нулевым временем просмотра
zero_a = (group_a == 0).sum()
zero_b = (group_b == 0).sum()
print(f"\nПользователи с нулевым временем просмотра:")
print(f"Группа A: {zero_a} ({zero_a/group_a_count*100:.1f}%)")
print(f"Группа B: {zero_b} ({zero_b/group_b_count*100:.1f}%)\n")

# 6 Вывод
print("Вывод\n")

print(f"1. Группа B показывает более высокое среднее время просмотра ({stats_b['mean']:.1f} мин против {stats_a['mean']:.1f} мин), следовательно алгоритм B более эфективен")
print(f"2. Алгоритм B более эффективен для верхних процентилях (95-й процентиль: +{np.percentile(group_b, 95)-np.percentile(group_a, 95):.1f} мин)")
print("3. Анализ процентилей оказался наиболее информативным, потому что:Выявляет неоднородность эффекта - средние значения искажены выбросами\n Показывает где работает алгоритм - на верхних, а не нижних процентилях \nУстойчив к выбросам - в данных есть экстремальные значения до 11,67 минут")
print("4. Рекомендации по запуску: Без доработки нежелательно запускать алгоритм для всех пользователей, так как он эфективен только для малого колличества самых активных пользователей,\n а для других может ухудшать опыт исползования")

Анализ A/B Теста: время просмотра

Dataset Summary:
Total users: 10000
Group A: 5000 users
Group B: 5000 users

Descriptive Statistics by Group:
        count  mean    std  min   25%  mediana   75%      max
Group                                                        
A     5000.00 56.35 262.31 0.00 14.73    26.83 44.95  8130.55
B     5000.00 71.48 408.52 0.00 15.38    27.35 46.35 11670.62
Key Percentiles by Group:
75й процентиль: A=44.9, B=46.4, Difference=1.4
90й процентиль: A=71.5, B=76.2, Difference=4.7
95й процентиль: A=119.3, B=153.6, Difference=34.3

Mann-Whitney U Test: p-value = 0.045342

Интерпретация:
Статистически значимое различие между группами (p < 0.05)

Пользователи с нулевым временем просмотра:
Группа A: 98 (2.0%)
Группа B: 102 (2.0%)

Вывод

1. Группа B показывает более высокое среднее время просмотра (71.5 мин против 56.3 мин), следовательно алгоритм B более эфективен
2. Алгоритм B более эффективен для верхних процентилях (95-й процентиль: +34.3 мин)
3. Анализ проце